# Getting the embeddings

> This notebook gets the embeddings (or latent space) from a multivariate time series 
given by a encoder (e.g., autoencoder)

In [4]:
from dvats.all import *
from tsai.data.preparation import SlidingWindow
from fastcore.all import *
import wandb
wandb_api = wandb.Api()
from yaml import load, FullLoader

## Config parameters

Put here everything that could be needed if this notebook

In [5]:
import utils.config as cfg
config, job_type = cfg.get_artifact_config_embeddings(True)

Current: /home/macu/work/nbs_pipeline
yml: ./config/03-embeddings.yaml
yml: ./config/03-embeddings.yaml
Getting content./config/03-embeddings.yaml
... About to replace includes with content
Load content./config/03-embeddings.yaml
enc_artifact: mi-santamaria/test-project/mvp:latest


## Run

In [6]:
run = wandb.init(
    entity      = config.wandb_entity,
    project     = config.wandb_project if config.use_wandb else 'work-nbs', 
    group       = config.wandb_group,
    job_type    = job_type,
    mode        = 'online' if config.use_wandb else 'disabled',
    anonymous   = 'never' if config.use_wandb else 'must',
    config      = config,
    resume      = 'allow'
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


In [7]:
# Botch to use artifacts offline
artifacts_gettr = run.use_artifact if config.use_wandb else wandb_api.artifact

Restore the encoder model and its associated configuration

In [8]:
enc_artifact = artifacts_gettr(config.enc_artifact, type='learner')

In [18]:
# TODO: This only works when you run it two timeS! WTF?
try:
    enc_learner = enc_artifact.to_obj()
except:
    enc_learner = enc_artifact.to_obj()
print(enc_learner)
print(enc_learner.version)

wandb:   1 of 1 files downloaded.  


<class 'AttributeError'>: 'InceptionTimePlus' object has no attribute 'version'

Restore the dataset artifact used for training the encoder. Even if we do not compute the dimensionality reduction over this dataset, we need to know the metadata of the encoder training set, to check that 
it matches with the dataset that we want to reduce.

In [10]:
enc_run = enc_artifact.logged_by()
enc_artifact_train = artifacts_gettr(enc_run.config['train_artifact'], type='dataset')
enc_artifact_train.name

'Semantic_Segmentation_TiltABP:v0'

Now we specify the dataset artifact that we want to get the embeddings from. If no 
artifact is defined, the artifact to reduce will be the one used for validate the encoder.

In [11]:
input_ar_name = ifnone(config.input_ar, 
                       f'{enc_artifact_train.entity}/{enc_artifact_train.project}/{enc_artifact_train.name}')
wandb.config.update({'input_ar': input_ar_name}, allow_val_change=True)
input_ar = artifacts_gettr(input_ar_name)
input_ar.name

'Semantic_Segmentation_TiltABP:v0'

In [12]:
df = input_ar.to_df()
df

wandb:   1 of 1 files downloaded.  


,time,abp
1970-01-01 00:00:00,0,6832.0
1970-01-01 00:00:01,1,6928.0
1970-01-01 00:00:02,2,6968.0
1970-01-01 00:00:03,3,6992.0
1970-01-01 00:00:04,4,6980.0
...,...,...
1970-01-01 11:06:35,39995,3468.0
1970-01-01 11:06:36,39996,3460.0
1970-01-01 11:06:37,39997,3456.0
1970-01-01 11:06:38,39998,3472.0


In [13]:
df.shape

(40000, 2)

In [14]:
enc_input, _ = SlidingWindow(window_len=enc_run.config['w'], 
                             stride=enc_run.config['stride'], 
                             get_y=[])(df)
enc_input.shape

(39971, 2, 30)

In [15]:
embs = get_enc_embs(enc_input, enc_learner, cpu=config.cpu, to_numpy=True)

In [16]:
if config.use_wandb: 
    run.log_artifact(ReferenceArtifact(embs, 'embeddings', metadata=dict(run.config)), 
                     aliases=f'run-{run.project}-{run.id}')

In [17]:
run.finish()